### General idea
1. Clean dataset => dfClean
2. Vectorize words => to probability density
3. Perform logistic regression on vectorized words 
    of scales of reviews 0 (0,.1),1 (.2,.3) ,2 (.4,.5),3 (.6,.7) ,4 (.8,.9, 1) reviews

In [4]:
# Read data set and stop words
import pandas as pd
import re 
import nltk
nltk.download('stopwords')
nltk.download('words')
rawData = pd.read_csv (r'.\AppReview.csv')
#df=df1.sample(n=1000)
len(rawData.index)



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\danxg\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\danxg\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


111143

In [5]:
rawData_v1=rawData[rawData["reviewerName"]!="A Google User"]
rawData_v1["reviewerRating"]=rawData["reviewerRating"]*10/2
rawData_v1=rawData_v1.astype({"reviewerRating": int})

len(rawData_v1.index)

C:\Users\danxg\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


90751

In [6]:
rawData_v1.groupby("reviewerRating").nunique()

,appID,reviewerName,reviewText,reviewerRating,reviewDate,textAnalytics
reviewerRating,,,,,,
1,329,7762,8156,1,2787,21
2,284,4011,4267,1,2195,17
3,322,6964,7311,1,2723,17
4,377,14009,14234,1,3287,23
5,470,49120,48225,1,3636,40


In [7]:
# frames = []
# for i in range(2,12,2):
#     frames.append(rawData_v1[rawData_v1["reviewerRating"]==i/10].sample(n=1000, replace=False))

# df=pd.concat(frames,ignore_index =True)
# len(df.index)
#df=rawData_v1.sample(n=5000)
#df.head()
#df.groupby("reviewerRating").size()
df=rawData_v1

In [8]:
df.shape[0]

90751

In [9]:
# Corpus of stop words
from nltk.corpus import stopwords
from nltk.corpus import words


In [10]:
# processes a review and returns a list of words
def review_to_words(review, string = True, remove_stopwords=True):
    # Remove HTML
    #review_text = BeautifulSoup(review).get_text()
    review_text=review
    # Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    # Convert words to lower case and split them
    thesewords = review_text.lower().split()
    # Ignore non=english words
#     englishWords = words.words()
#     thesewords = [w for w in thesewords if w in englishWords]

    
    # Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        thesewords = [w for w in thesewords if not w in stops]
    if string:
        return " ".join(thesewords)
    else:
        return thesewords

In [11]:
#Clean up text
#Remove non-ascii text
#Remove all rows missing reviewerName
def fixString(x):
    return x.encode('ascii',errors='ignore')

# df[["reviewText"]]=df[["reviewText"]].apply(lambda x: str(x["reviewText"]).encode('ascii',errors='ignore').decode(), axis=1)
# df[["reviewerName"]]=df[["reviewerName"]].apply(lambda x: str(x["reviewerName"]).encode('ascii',errors='ignore').decode(), axis=1)
df[["reviewText"]]=df[["reviewText"]].apply(lambda x: review_to_words(x["reviewText"]), axis=1)
df[["reviewerName"]]=df[["reviewerName"]].apply(lambda x: str(x["reviewerName"]).encode('ascii',errors='ignore').decode(), axis=1)

dfCleaned=df[df['reviewText'].str.strip().astype(bool)]
dfCleaned=dfCleaned[df['reviewerName'].str.strip().astype(bool)]

#
#dfCleaned

In [12]:
df=dfCleaned
import numpy as np
df[df['reviewerRating'] != 3]
df['Positivity'] = np.where(df['reviewerRating'] > 3, 1, 0)
cols = ['appID', 'reviewerName', 'reviewerRating', 'reviewDate', 'textAnalytics']
df.drop(cols, axis=1, inplace=True)
df.dropna(inplace=True)
df.head()

,reviewText,Positivity
0,love well worth money full version came ad blo...,1
1,awful bug allow use space bar want type search...,0
2,would stars except bugs example incognito tab ...,1
3,worked perfect weeks ago browsing experience s...,0
4,hands best browser play store even flagship de...,1


## Vectorize words
### This is based on 
https://towardsdatascience.com/sentiment-analysis-a-how-to-guide-with-movie-reviews-9ae335e6bcb2
With actual logistic regression:
https://towardsdatascience.com/sentiment-classification-with-logistic-regression-analyzing-yelp-reviews-3981678c3b44


In [13]:
# # import statements
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.feature_extraction.text import TfidfTransformer
# import scipy
# from sklearn.model_selection import train_test_split
# from nltk.corpus import words

# dfReviews = dfCleaned['reviewText']
# dfTrain, dfTest = train_test_split(dfReviews, test_size=.1)

# # Initialize a bag of words
# #vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None, stop_words = None, max_features = 5000) 
# vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None, \
#                              stop_words = None, max_features = 1000) 

# # Fit transform the data 
# train_feat = vectorizer.fit_transform(dfTrain).toarray()
# test_feat = vectorizer.transform(dfTest).toarray()


# # TFIDF train set
# tfidf_transformer = TfidfTransformer().fit(train_feat)
# train_tfidf = tfidf_transformer.transform(train_feat)
 
# # apply tfidf to test set
# test_tfidf = tfidf_transformer.transform(test_feat)

# train_tfidf.todense()


# import statements
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
tfidf.fit(dfCleaned["reviewText"])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [16]:
dfCleaned.iloc[1]["reviewText"]
X = tfidf.transform(dfCleaned["reviewText"])
print([X[1, tfidf.vocabulary_['awful']]])

[0.4528846823706622]


### Modelling part
1. Leverage the raw vector count and the tf-idf weighted version

In [36]:
# Get trained Y data and test Y data
trainYdata = pd.merge(dfTrain.to_frame(), dfCleaned[['reviewerRating']], left_index=True, right_index=True)
testYdata = pd.merge(dfTest.to_frame(), dfCleaned[['reviewerRating']], left_index=True, right_index=True)

In [37]:
trainYdata

,reviewText,reviewerRating
51034,good full review,1.0
63932,ar full review,1.0
14014,play back speed happy could speed great reason...,0.2
75130,log log see fix load nearby setting full review,0.2
54571,great editor editor used great color please ad...,0.8
...,...,...
42258,helpful full review,1.0
25522,marking recent read click article go back mark...,0.4
46438,simple nice good highly full review,1.0
87306,open source decent file manager full review,0.8


In [38]:
# Get trained X data and test X data
trainXdata = train_tfidf.todense()
testXdata = test_tfidf.todense()
trainXdata

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [41]:
#from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression

In [104]:
# Train data - map the y to ints of scales of reviews 0,1,2,3,4 reviews
y = trainYdata[['reviewerRating']]
#y_int = trainYdata['reviewerRating'].apply(lambda x: 0 if x<.2 else (1 if x<.4 else (2 if x<.6 else (3 if x<.8 else 4))))
#y_int = trainYdata['reviewerRating'].apply(lambda x: x*10)
y_int = trainYdata['reviewerRating'].apply(lambda x: 0 if x <.5 else 1)

X = trainXdata

#clf = LogisticRegression(random_state=0, max_iter=1000, multi_class='auto', class_weight= 'balanced').fit(X, y_int)

clf = LogisticRegression(random_state=0, max_iter=1000, multi_class='auto').fit(X, y_int)

# Check trained accuracy
clf.predict(X[:20, :])
clf.predict_proba(X[:20, :])
clf.score(X, y_int)
#clf


0.8935599284436494

In [96]:
from sklearn import model_selection
kfold = model_selection.KFold(n_splits=10, random_state=7, shuffle=True)
scoring = 'accuracy'
results = model_selection.cross_val_score(clf, trainXdata, y_int, cv=kfold, scoring=scoring)
print("Accuracy: %.3f (%.3f)" % (results.mean(), results.std()))

Accuracy: 0.814 (0.021)


In [97]:
# # look at some more data
# import scipy.sparse

# featureNamesList=vectorizer.get_feature_names()
# type(train_tfidf)

# cx = scipy.sparse.coo_matrix(train_tfidf)

# print(dfTrain.iloc[0])
# for i,j,v in zip(cx.row, cx.col, cx.data):
#     if i!=0:
#         break
#     print("(%d, %d), %s = %s" % (i,j,featureNamesList[j], v))

In [98]:
#trainYdata[0:2]

In [99]:
#clf.predict_proba(X[:2, :])

In [100]:
#X[:2, :]

### Check test accuracy

In [101]:
#y_int_test = testYdata['reviewerRating'].apply(lambda x: 0 if x<.2 else (1 if x<.4 else (2 if x<.6 else (3 if x<.8 else 4))))
#y_int_test = testYdata['reviewerRating'].apply(lambda x: x*10)
y_int_test = testYdata['reviewerRating'].apply(lambda x: 0 if x<.5 else 1)

In [102]:
clf.predict(testXdata)
y_pred = clf.predict_proba(testXdata)
clf.score(testXdata, y_int_test)
y_true = y_int_test

In [103]:
import numpy as np
from sklearn.metrics import confusion_matrix
y_pred_values = [y_pred[:,[1]] > .5][0] + 0
y_true_values=y_true.to_numpy()
y_pred_values=np.concatenate(y_pred_values).ravel()
confusion_matrix = confusion_matrix(y_true_values, y_pred_values)

row_sums = confusion_matrix.sum(axis=1)
new_matrix = confusion_matrix / row_sums[:, np.newaxis]
new_matrix

array([[0.65116279, 0.34883721],
       [0.15085158, 0.84914842]])

In [53]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 100, random_state = 42, max_depth = 10, n_jobs=-1)
# Train the model on training data
clt = rf.fit(X, y_int);



In [56]:
clt.predict(testXdata)

clt.score(testXdata, y_int_test)

0.21220939279409368

In [54]:
clf.predict(X[:2, :])
clf.predict_proba(X[:2, :])
clf.score(X, y_int)


clf.predict(testXdata)
clf.predict_proba(testXdata)
clf.score(testXdata, y_int_test)

0.8875502008032129